In [1]:
import pandas as pd
import networkx as nx
import numpy as np
np.random.seed(357823)
import random
random.seed(357823)

Primero, vamos a cargar los datos de los nodos y las aristas.

In [2]:
edges=pd.read_csv('edges.csv')
vertices=pd.read_csv('nodes.csv')

In [3]:
edges.head()

,source,target
0,Baron Zemo,Blackout (Marcus Daniels)
1,Baron Zemo,Vermin (comics)
2,N'astirh,S'ym
3,N'astirh,Belasco (Marvel Comics)
4,N'astirh,Madelyne Pryor


In [4]:
vertices.head()

,group,id,size
0,1,Baron Zemo,2
1,1,N'astirh,3
2,0,Silver Sable,1
3,0,Hope Summers (comics),1
4,0,Magik (Illyana Rasputin),1


In [5]:
G=nx.from_pandas_edgelist(edges,source='source',target='target')

Métricas relacionales para cada nodo.

In [6]:
from collections import Counter
degree=nx.degree(G)
triangles=nx.triangles(G)
degree_centrality= nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering = nx.clustering(G)
tam_comp = {}
maximo=max(len(c) for c in nx.connected_components(G))
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        tam_comp[node] = len(component)
tam=Counter((len(c) for c in nx.connected_components(G)))
vertices = vertices[vertices['id'].map(lambda v: tam_comp[v] >= 10)]
print(tam)



Counter({2: 30, 3: 11, 4: 5, 5: 4, 6: 3, 7: 1, 181: 1, 11: 1})


In [7]:
degree_dict= {d[0]:d[1] for d in degree}
degreeL = [degree[v["id"]]for _,v in vertices.iterrows()]
trianglesL=[triangles[v['id']] for _,v in vertices.iterrows()]
degree_centralityL = [degree_centrality[v['id']] for _,v in vertices.iterrows()]
closeness_centralityL = [closeness_centrality[v['id']] for _,v in vertices.iterrows()]
betweenness_centralityL = [betweenness_centrality[v['id']] for _,v in vertices.iterrows()]
eigenvector_centralityL =[eigenvector_centrality[v['id']] for _,v in vertices.iterrows()]
clusteringL = [clustering[v['id']] for _,v in vertices.iterrows()]
# component_idL= [component_id[v['id']] for _,v in vertices.iterrows()]

Crear el dataframe con los datos.

In [8]:
df = pd.DataFrame({'triangle':trianglesL, "degree_centrality":degree_centralityL,
                  'closeness_centrality':closeness_centralityL, 'betweenness_centrality':betweenness_centralityL,
                    'eigenvector_centrality':eigenvector_centralityL,'clustering':clusteringL,'group':vertices['group']})



In [9]:
df.head()

,triangle,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering,group
3,0,0.002865,0.065194,0.000000,1.361871e-03,0.000000,0
6,0,0.002865,0.009551,0.000000,1.666953e-17,0.000000,1
9,0,0.017192,0.110388,0.176275,1.224383e-01,0.000000,1
14,0,0.002865,0.052361,0.000000,3.509777e-05,0.000000,2
16,1,0.008596,0.070706,0.011593,6.506930e-04,0.333333,1


In [10]:
atributos=df.iloc[:,0:-1]
objetivo=df.iloc[:,-1]

In [11]:
from sklearn.model_selection import train_test_split
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,random_state=357823)

In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer(
    [
        ('normalizador', MinMaxScaler(), list(range(0, 6)))
    ]
)

In [44]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'arbol__max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'arbol__min_samples_split': range(2, 20)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('arbol',DecisionTreeClassifier(random_state=357823))])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold
                                   )
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('arbol',
                                        DecisionTreeClassifier(random_state=357823))]),
             param_grid={'arbol__max_depth': range(3, 11),
                         'arbol__min_samples_split': range(2, 20)},
             scoring='accuracy')

In [45]:
búsqueda_en_rejilla.cv_results_

{'mean_fit_time': array([0.00350602, 0.00240314, 0.00254793, 0.00238936, 0.00315313,
        0.00252845, 0.00311527, 0.00276687, 0.00257807, 0.00270426,
        0.00311289, 0.00272932, 0.00290713, 0.00259838, 0.00256071,
        0.00280099, 0.00284157, 0.00413659, 0.00500665, 0.00305901,
        0.00288436, 0.00266547, 0.00294559, 0.0028527 , 0.00274937,
        0.0028784 , 0.00311503, 0.00261405, 0.00269079, 0.0031759 ,
        0.00476279, 0.00424497, 0.00437276, 0.00391622, 0.00384345,
        0.00383365, 0.0029983 , 0.00282857, 0.00339563, 0.00316217,
        0.0032495 , 0.00277104, 0.0031739 , 0.00305679, 0.0029532 ,
        0.00353594, 0.00329826, 0.00294051, 0.00295553, 0.00274816,
        0.00294406, 0.00293314, 0.00283337, 0.00482066, 0.00265498,
        0.0029387 , 0.00244684, 0.00278752, 0.00279427, 0.00299089,
        0.00264361, 0.00294118, 0.00288892, 0.00297086, 0.00359318,
        0.00330577, 0.00249646, 0.00278285, 0.00284836, 0.00251129,
        0.00294623, 0.00311577,

In [46]:
print(búsqueda_en_rejilla.best_score_)

0.7262500000000001


In [50]:

from sklearn.metrics import accuracy_score,classification_report
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))


0.717948717948718
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.81      0.77      0.79        22
           2       0.33      0.25      0.29         4

    accuracy                           0.72        39
   macro avg       0.60      0.60      0.60        39
weighted avg       0.71      0.72      0.71        39



In [49]:
print(classification_report(objetivo_entrenamiento,modelo_arbol.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89        51
           1       0.91      0.97      0.94        86
           2       0.75      0.56      0.64        16

    accuracy                           0.90       153
   macro avg       0.85      0.80      0.82       153
weighted avg       0.89      0.90      0.89       153



<h1>KNN (K-Nearest Neighbors)<h1>

Este algoritmo realiza una predicción a partir del valor de sus k instancias más cercanas. En este caso, utilizaremos la distancia euclidea para medir la distacia entre las instancias.
Usaremos una validación cruzada estratificada con 10 particiones.

In [51]:
from sklearn.neighbors import KNeighborsClassifier
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'knn__metric': ["euclidean","manhattan"],
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'knn__n_neighbors': range(1, 11)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('knn',KNeighborsClassifier())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(1, 11)},
             scoring='accuracy')

In [52]:
búsqueda_en_rejilla.best_score_

np.float64(0.6545833333333333)

Ahora vamos comprobar los resultados, para ello buscamos el mejor modelo encontrado en la busqueda de hiperparametros. Para ver los resultados utilizaremos la función classification_report() que nos muestra un reporte detallado con los datos que nos interesan, en esta celda lo haremos con el conjunto de prueba.

In [53]:
modelo_knn=búsqueda_en_rejilla.best_estimator_
print(accuracy_score(objetivo_prueba,modelo_knn.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_knn.predict(atributos_prueba)))

0.5897435897435898
              precision    recall  f1-score   support

           0       0.50      0.54      0.52        13
           1       0.78      0.64      0.70        22
           2       0.29      0.50      0.36         4

    accuracy                           0.59        39
   macro avg       0.52      0.56      0.53        39
weighted avg       0.63      0.59      0.61        39



En esta celda imprimiremos el reporte del conjunto de entrenamiento. 

In [54]:
print(classification_report(objetivo_entrenamiento,modelo_knn.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        51
           1       1.00      0.99      0.99        86
           2       0.89      1.00      0.94        16

    accuracy                           0.99       153
   macro avg       0.96      0.99      0.98       153
weighted avg       0.99      0.99      0.99       153



In [61]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Normalization,Dense
from keras import Sequential, Input
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam
import tensorflow as tf
import os
tf.random.set_seed(48679)
os.environ['PYTHONHASHSEED'] = '48679'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [62]:
enc=OneHotEncoder()
objetivo_entrenamiento_cod=enc.fit_transform(objetivo_entrenamiento.to_frame()).toarray()
weights=compute_class_weight('balanced',
                     classes=np.array([0, 1,2]),
                     y=objetivo_entrenamiento)
print(weights)

red = Sequential()
normalizador_red=Normalization()
normalizador_red.adapt(atributos_entrenamiento.to_numpy())
red.add(Input(shape=(6,)))
red.add(normalizador_red)
red.add(Dense(50,activation="relu"))
red.add(Dense(10,activation="tanh"))
red.add(Dense(3,activation="softmax"))
red.compile(optimizer=Adam(learning_rate=0.05), loss='categorical_crossentropy',
                   metrics=['accuracy'])
red.fit(atributos_entrenamiento,objetivo_entrenamiento_cod,batch_size=256,epochs=300)

[1.         0.59302326 3.1875    ]
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step - accuracy: 0.3203 - loss: 1.2133
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5556 - loss: 0.9270
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5621 - loss: 0.9427
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5621 - loss: 0.9212
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5556 - loss: 0.9016
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5686 - loss: 0.8821
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5752 - loss: 0.8708
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5752 - loss: 0.8671
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5817 - loss: 0.8563
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5948 - loss: 0.8442
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6275 - loss: 0.8285
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [63]:
pred=np.argmax(red.predict(atributos_prueba),axis=1)
print(classification_report(objetivo_prueba,pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
              precision    recall  f1-score   support

           0       0.67      0.92      0.77        13
           1       0.89      0.77      0.83        22
           2       0.50      0.25      0.33         4

    accuracy                           0.77        39
   macro avg       0.69      0.65      0.65        39
weighted avg       0.78      0.77      0.76        39



In [66]:
pred_ent=np.argmax(red.predict(atributos_entrenamiento),axis=1)
print(classification_report(objetivo_entrenamiento,pred_ent))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
              precision    recall  f1-score   support

           0       0.82      0.96      0.88        51
           1       0.96      0.90      0.93        86
           2       0.92      0.75      0.83        16

    accuracy                           0.90       153
   macro avg       0.90      0.87      0.88       153
weighted avg       0.91      0.90      0.90       153



In [55]:
from sklearn.preprocessing import KBinsDiscretizer
discretizador = ColumnTransformer([('discretizador',
                                    KBinsDiscretizer(
    n_bins=4, 
    encode='ordinal',  
    strategy='uniform'  
),
                                   list(range(0,6)))])

In [56]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
   
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'naive_bayes__alpha': range(1, 11)
    
}
kfold = StratifiedKFold(n_splits=5)
tuberia_arbol=Pipeline([('discretizador',discretizador),('naive_bayes',CategoricalNB())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('discretizador',
                                        ColumnTransformer(transformers=[('discretizador',
                                                                         KBinsDiscretizer(encode='ordinal',
                                                                                          n_bins=4,
                                                                                          strategy='uniform'),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('naive_bayes', CategoricalNB())]),
             param_grid={'naive_bayes__alpha': range(1, 11)},
             scoring='accuracy')

In [57]:
búsqueda_en_rejilla.best_score_

np.float64(0.5492473118279569)

In [59]:
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.55      0.95      0.70        22
           2       0.00      0.00      0.00         4

    accuracy                           0.54        39
   macro avg       0.18      0.32      0.23        39
weighted avg       0.31      0.54      0.39        39



c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [60]:
print(classification_report(objetivo_entrenamiento,modelo_arbol.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       1.00      0.06      0.11        51
           1       0.57      0.99      0.72        86
           2       0.00      0.00      0.00        16

    accuracy                           0.58       153
   macro avg       0.52      0.35      0.28       153
weighted avg       0.65      0.58      0.44       153

